<a href="https://colab.research.google.com/github/Marisagf/Projetos_de_Ciencias_de_Dados/blob/main/Previs%C3%A3o_de_Sobreviv%C3%AAncia_ao_C%C3%A2ncer_de_Mama.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **Previsão de Sobrevivência ao Câncer de Mama**

 Com o avanço das tecnologias e o aumento da disponibilidade de dados na área da saúde, o uso de aprendizado de máquina se tornou uma ferramenta poderosa para apoiar decisões clínicas e salvar vidas.

Um exemplo prático disso é a possibilidade de prever a sobrevivência de pacientes com doenças graves, como o câncer de mama, utilizando algoritmos que analisam padrões nos dados clínicos.

Neste estudo, utilizamos um conjunto de dados reais com informações de pacientes com câncer de mama para treinar um modelo capaz de prever se um paciente tem maior probabilidade de sobrevivência (vivo) ou não (óbito), com base em características como idade, estágio do câncer, tratamentos realizados, entre outros.

📌 Fonte dos dados: Kaggle
🔗 Breast Cancer Dataset – AmandaM1



#### *Conjunto de Dados*

Este conjunto de dados consiste em um grupo de pacientes com câncer de mama que passaram por cirurgia para remover o tumor. O conjunto de dados é composto pelas seguintes variáveis:

    - Patient_ID: identificador único do paciente
    - Age: idade no momento do diagnóstico (anos)
    - Gender: Masculino/Feminino
    - Protein1, Protein2, Protein3, Protein4: níveis de expressão (unidades não definidas)
    - Tumour_Stage: I, II, III
    - Histology: Carcinoma Ductal Infiltrante, Carcinoma Lobular Infiltrante, Carcinoma Mucinoso
    - ER status: Positivo/Negativo
    - PR status: Positivo/Negativo
    - HER2 status: Positivo/Negativo
    - Surgery_type: Lumpectomia, Mastectomia Simples, Mastectomia Radical Modificada, Outro
    - Date_of_Surgery: Data em que a cirurgia foi realizada (em DD-MON-YY)
    - Date_of_Last_Visit: Data da última visita (em DD-MON-YY) [pode ser nulo, caso o paciente não tenha retornado após a cirurgia]
    - Patient_Status: Vivo/Morto [pode ser nulo, caso o paciente não tenha retornado após a cirurgia e não haja informação disponível sobre se o paciente está vivo ou morto].



#### *Importação das bibliotecas / pacotes necessários em nosso projeto*

In [ ]:
# Os nossos pacotes tradicionais
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# Os pacotes para normalização e preparação dos dados
import sklearn  #para identificação da versão
from sklearn.preprocessing import LabelEncoder, OrdinalEncoder
from sklearn.model_selection import train_test_split

# Os pacotes destinados a aprendizagem de máquina (algoritmos)
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Importando o método SMOTE da biblioteca imblearn
from imblearn.over_sampling import SMOTE

# Dentro da biblioteca <'imbalanced-learn'> existe um método <'SMOTE'> que significa: Synthetic Minority Over-sampling Technique.
# Ou seja... O SMOTE é uma técnica de oversampling que cria novas instâncias sintéticas da classe minoritária no conjunto de dados,
# a fim de balancear a distribuição das classes.

# Os pacotes destinados a report de métricas dos modelos
from sklearn.metrics import roc_curve, roc_auc_score, confusion_matrix, accuracy_score, classification_report

# para evitar mensagens de alerta/warnings.
import warnings
warnings.filterwarnings("ignore")

# Carregar o módulo de funções para limpeza de dados
from limpeza_dados import *